In [1]:
!pip install datasets bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [51]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    roc_auc_score,
    precision_recall_curve,
    auc,
)
from sklearn.utils import resample

In [52]:
# Ensure reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Load the pre-trained model and tokenizer without fine-tuning
model_name = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [54]:
# Load the test data
test_data = pd.read_pickle("aggregated_test_df_cleaned.pkl")

# Preprocess the test data
def preprocess_patient_data(row):
    gender = "male" if row["gender"] == "M" else "female"
    age = f"{row['age']} years old"
    diagnoses = "none" if not row["diagnoses"] else ", ".join(row["diagnoses"])
    abnormal_labs = "none" if not row["abnormal_labs"] else ", ".join(row["abnormal_labs"])
    formatted_history = (
        f"The patient is a {age} {gender}. "
        f"Diagnoses include {diagnoses}. "
        f"Abnormal lab tests include {abnormal_labs}."
    )
    return formatted_history

test_data["formatted_history"] = test_data.apply(preprocess_patient_data, axis=1)
test_texts = test_data["formatted_history"].tolist()
test_labels = test_data["is_ckd"].tolist()

In [55]:
# Tokenize the test data
def tokenize_texts(texts):
    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )

test_encodings = tokenize_texts(test_texts)

In [56]:
# Create a custom Dataset and DataLoader
class TestDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels  # Optional

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

In [57]:
# Instantiate the dataset and dataloader
test_dataset = TestDataset(test_encodings, test_labels)
batch_size = 8  # Adjust according to your GPU memory
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Perform predictions
all_logits = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        all_logits.append(logits.cpu())
        all_labels.append(labels.cpu())

In [58]:
# Concatenate all logits and labels
all_logits = torch.cat(all_logits)
all_labels = torch.cat(all_labels)

# Compute probabilities and predictions
probs = torch.softmax(all_logits, dim=1)
predictions = torch.argmax(probs, dim=1)

# Convert to numpy arrays
probs_np = probs.numpy()
predictions_np = predictions.numpy()
labels_np = all_labels.numpy()

# Compute evaluation metrics
precision, recall, f1, _ = precision_recall_fscore_support(
    labels_np, predictions_np, average="binary"
)
acc = accuracy_score(labels_np, predictions_np)
roc_auc = roc_auc_score(labels_np, probs_np[:, 1])

# Compute PR AUC
precision_curve, recall_curve, _ = precision_recall_curve(labels_np, probs_np[:, 1])
pr_auc = auc(recall_curve, precision_curve)

# Print evaluation metrics
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print(f"PR AUC: {pr_auc:.4f}")

Accuracy: 0.1075
Precision: 0.1075
Recall: 1.0000
F1 Score: 0.1941
ROC AUC: 0.4689
PR AUC: 0.0980


In [59]:
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, auc
from sklearn.utils import resample
import numpy as np

n_bootstraps = 1000
roc_bootstrapped_scores = []
pr_bootstrapped_scores = []

for i in range(n_bootstraps):
    # Bootstrap by sampling with replacement on the prediction indices
    indices = resample(
        np.arange(len(labels_np)), replace=True, n_samples=len(labels_np), random_state=i
    )
    if len(np.unique(labels_np[indices])) < 2:
        # Skip this sample because it doesn't have at least one positive and one negative class
        continue

    # Compute ROC AUC for the current bootstrap sample
    roc_score = roc_auc_score(labels_np[indices], probs_np[indices, 1])
    roc_bootstrapped_scores.append(roc_score)

    # Compute PR AUC for the current bootstrap sample
    precision_curve, recall_curve, _ = precision_recall_curve(labels_np[indices], probs_np[indices, 1])
    pr_score = auc(recall_curve, precision_curve)
    pr_bootstrapped_scores.append(pr_score)

# Convert lists to numpy arrays
roc_bootstrapped_scores = np.array(roc_bootstrapped_scores)
pr_bootstrapped_scores = np.array(pr_bootstrapped_scores)

# Sort scores
sorted_roc_scores = np.sort(roc_bootstrapped_scores)
sorted_pr_scores = np.sort(pr_bootstrapped_scores)

# Compute 95% confidence intervals
roc_confidence_lower = sorted_roc_scores[int(0.025 * len(sorted_roc_scores))]
roc_confidence_upper = sorted_roc_scores[int(0.975 * len(sorted_roc_scores))]

pr_confidence_lower = sorted_pr_scores[int(0.025 * len(sorted_pr_scores))]
pr_confidence_upper = sorted_pr_scores[int(0.975 * len(sorted_pr_scores))]

# Print confidence intervals
print(f"95% Confidence Interval for ROC AUC: [{roc_confidence_lower:.4f}, {roc_confidence_upper:.4f}]")
print(f"95% Confidence Interval for PR AUC: [{pr_confidence_lower:.4f}, {pr_confidence_upper:.4f}]")

95% Confidence Interval for ROC AUC: [0.4531, 0.4835]
95% Confidence Interval for PR AUC: [0.0919, 0.1044]


ROC AUC:
0.4683±0.0152
PR AUC:
0.0982±0.0063